![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/6.Clinical_Context_Spell_Checker.ipynb)

# 6. Context Spell Checker - Medical


In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.1.0
Spark NLP_JSL Version : 5.1.0


## spellcheck_clinical

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = RecursiveTokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")\
    .setPrefixes(["\"", "(", "[", "\n"])\
    .setSuffixes([".", ",", "?", ")","!", "'s"])

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("token")\
    .setOutputCol("checked")

spellcheck_clinical download started this may take some time.
Approximate size to download 134.7 MB
[OK!]


In [ ]:
pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel
])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = LightPipeline(pipeline.fit(empty_ds))

Ok!, at this point we have our spell checking pipeline as expected. Let's see what we can do with it, see these errors,

_She was **treathed** with a five day course of **amoxicilin** for a **resperatory** **truct** infection._

_With pain well controlled on **orall** **meditation**, she was discharged to **reihabilitation** **facilitay**._


_Her **adominal** examination is soft, nontender, and **nonintended**_

_The patient was seen by the **entocrinology** service and she was discharged on 40 units of **unsilin** glargine at night_
      
_No __cute__ distress_

Check that some of the errors are valid English words, only by considering the context the right choice can be made.

In [ ]:
example = ["She was treathed with a five day course of amoxicilin for a resperatory truct infection . ",
           "With pain well controlled on orall meditation, she was discharged to reihabilitation facilitay.",
           "Her adominal examination is soft, nontender, and nonintended.",
           "The patient was seen by the entocrinology service and she was discharged on 40 units of unsilin glargine at night",
           "No cute distress",
          ]

for pairs in lp.annotate(example):
    print(list(zip(pairs['token'],pairs['checked'])))

[('She', 'She'), ('was', 'was'), ('treathed', 'treated'), ('with', 'with'), ('a', 'a'), ('five', 'five'), ('day', 'day'), ('course', 'course'), ('of', 'of'), ('amoxicilin', 'amoxicillin'), ('for', 'for'), ('a', 'a'), ('resperatory', 'respiratory'), ('truct', 'tract'), ('infection', 'infection'), ('.', '.')]
[('With', 'With'), ('pain', 'pain'), ('well', 'well'), ('controlled', 'controlled'), ('on', 'on'), ('orall', 'oral'), ('meditation', 'medication'), (',', ','), ('she', 'she'), ('was', 'was'), ('discharged', 'discharged'), ('to', 'to'), ('reihabilitation', 'rehabilitation'), ('facilitay', 'facility'), ('.', '.')]
[('Her', 'Her'), ('adominal', 'abdominal'), ('examination', 'examination'), ('is', 'is'), ('soft', 'soft'), (',', ','), ('nontender', 'nontender'), (',', ','), ('and', 'and'), ('nonintended', 'nondistended'), ('.', '.')]
[('The', 'The'), ('patient', 'patient'), ('was', 'was'), ('seen', 'seen'), ('by', 'by'), ('the', 'the'), ('entocrinology', 'endocrinology'), ('service', 'se

In [ ]:
print("Corrected tokens:\n")

pair_list = [list(zip(pairs['token'],pairs['checked'])) for pairs in lp.annotate(example)]
corrected_list = [i for pair in pair_list for i in pair if i[0] != i[1]]
corrected_list

Corrected tokens:



[('treathed', 'treated'),
 ('amoxicilin', 'amoxicillin'),
 ('resperatory', 'respiratory'),
 ('truct', 'tract'),
 ('orall', 'oral'),
 ('meditation', 'medication'),
 ('reihabilitation', 'rehabilitation'),
 ('facilitay', 'facility'),
 ('adominal', 'abdominal'),
 ('nonintended', 'nondistended'),
 ('entocrinology', 'endocrinology'),
 ('unsilin', 'insulin'),
 ('cute', 'acute')]

## spellcheck_drug_norvig

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

spell = NorvigSweetingModel.pretrained("spellcheck_drug_norvig", "en", "clinical/models")\
    .setInputCols("token")\
    .setOutputCol("corrected_token")\

pipeline = Pipeline(
    stages = [
        documentAssembler,
        tokenizer,
        spell
])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = LightPipeline(pipeline.fit(empty_ds))

spellcheck_drug_norvig download started this may take some time.
Approximate size to download 4.3 MB
[OK!]


In [ ]:
example = ["You have to take Amrosia artemisiifoli , Oactra and a bit of Grastk and lastacaf ",
          ]

for pairs in lp.annotate(example):
    print(list(zip(pairs['token'],pairs['corrected_token'])))

[('You', 'You'), ('have', 'have'), ('to', 'to'), ('take', 'take'), ('Amrosia', 'Ambrosia'), ('artemisiifoli', 'artemisiifolia'), (',', ','), ('Oactra', 'Odactra'), ('and', 'and'), ('a', 'a'), ('bit', 'bit'), ('of', 'of'), ('Grastk', 'Grastek'), ('and', 'and'), ('lastacaf', 'lastacaft')]


In [ ]:
print("Corrected tokens:\n")

pair_list = [list(zip(pairs['token'],pairs['corrected_token'])) for pairs in lp.annotate(example)]
corrected_list = [i for pair in pair_list for i in pair if i[0] != i[1]]
corrected_list

Corrected tokens:



[('Amrosia', 'Ambrosia'),
 ('artemisiifoli', 'artemisiifolia'),
 ('Oactra', 'Odactra'),
 ('Grastk', 'Grastek'),
 ('lastacaf', 'lastacaft')]